In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import pseudopeople as psp
import os
from dask.distributed import LocalCluster

!date
!whoami
!uname -a
!pwd
!python --version
!conda info --envs | grep '\*'
!conda list | grep -e pandas -e numpy -e vivarium -e pseudopeople -e dask
!pip freeze | grep pseudopeople

Tue 21 May 2024 02:20:39 PM PDT
ndbs
Linux long-slurm-sarchive-p0081 5.4.0-176-generic #196-Ubuntu SMP Fri Mar 22 16:46:39 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/pseudopeople_testing
Python 3.11.9
pppl311dask           *  /ihme/homes/ndbs/miniconda3/envs/pppl311dask
# packages in environment at /ihme/homes/ndbs/miniconda3/envs/pppl311dask:
dask                      2024.5.1                 pypi_0    pypi
dask-expr                 1.1.1                    pypi_0    pypi
numpy                     1.26.4                   pypi_0    pypi
pandas                    2.2.2                    pypi_0    pypi
pseudopeople              1.0.1.dev15+g6234814          pypi_0    pypi
vivarium                  2.3.7                    pypi_0    pypi
-e git+https://github.com/ihmeuw/pseudopeople.git@6234814722ea3cc1e66f8ecd7ab82708bf3170b6#egg=pseudopeople


# Set up a local Dask cluster on the current Slurm node

In [18]:
!echo $SLURM_CPUS_ON_NODE

2


In [19]:
!echo $SLURM_MEM_PER_NODE

20480


In [2]:
cluster = LocalCluster(
  n_workers=int(os.environ["SLURM_CPUS_ON_NODE"]),
  threads_per_worker=1,
  memory_limit=(
    # Per worker!
    int(os.environ["SLURM_MEM_PER_NODE"]) / int(os.environ["SLURM_CPUS_ON_NODE"])
  ) * 1_000 * 1_000, # Dask uses bytes, Slurm reports in megabytes.
)
client = cluster.get_client() # NOTE: This step is necessary, even if you don't use "client"!

# Define data directories

In [3]:
project_path = Path('/mnt/team/simulation_science/pub/models/vivarium_census_prl_synth_pop')
usa_path = project_path / 'results/release_02_yellow/full_data/united_states_of_america'

# Paths to latest zipped data
last_zipped_sim_path = usa_path / '2023_08_21_16_35_27'
last_zipped_path = last_zipped_sim_path / 'final_results/2023_08_31_15_58_01'

ri_data_dir = last_zipped_path / 'states/pseudopeople_simulated_population_rhode_island_2_0_0'
usa_data_dir = last_zipped_path / 'pseudopeople_simulated_population_usa_2_0_0'

# Generate some data!

In [4]:
%%time
usa_acs = psp.generate_american_community_survey(usa_data_dir, engine='dask')
usa_acs

CPU times: user 6.24 s, sys: 2.23 s, total: 8.47 s
Wall time: 1min 6s


,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity
npartitions=334,,,,,,,,,,,,,,,,,,
,object,object,datetime64[ns],object,object,object,object,object,object,object,object,object,category[unknown],object,object,category[unknown],category[unknown],category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [5]:
%%time
usa_acs.head()

2024-05-21 14:21:55.448 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function leave_blank on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.


CPU times: user 194 ms, sys: 104 ms, total: 298 ms
Wall time: 3.99 s


2024-05-21 14:21:56.135 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function make_typos on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.


,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity
1,28_31572,28_12763,2020-01-28,Nia,B,Gomez,12,03/26/2007,51,stancliff rd,NaN,farmington,MN,55104,Household,Biological child,Female,Latino
3,28_66965,28_27014,2020-01-28,Kathleen,E,Nies,62,03/31/1957,2513,broadway street,NaN,florence,MS,38732,Household,Reference person,Female,White
4,28_103770,28_41938,2020-01-28,David,A,Wilson,67,05/07/1952,810,braxton pl,NaN,fruita,CO,80016,Household,Reference person,Male,Black
7,28_107747,28_43578,2020-01-28,Jessica,J,Palmer,35,08/15/1984,1000,aintree ct,NaN,ardmore,OK,73439,Household,Opposite-sex spouse,Female,White
10,28_138628,28_55890,2020-01-28,Rose,L,Mckenney,83,07/19/1936,405,ne tillamook st,NaN,jacksonville,FL,NaN,Household,Reference person,Female,White


In [6]:
%%time
people_named_jeff = usa_acs[usa_acs.first_name == "Jeff"]
people_named_jeff_pandas = people_named_jeff.compute()

2024-05-21 14:22:03.860 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function leave_blank on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 14:22:04.156 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function write_wrong_digits on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 14:22:05.885 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function make_ocr_errors on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 14:22:06.055 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function make_typos on column unit_number. This is likely due to a combination of the conf

CPU times: user 12.9 s, sys: 7.02 s, total: 19.9 s
Wall time: 3min 54s


In [11]:
%%time
people_named_jeff_pandas

CPU times: user 12 µs, sys: 7 µs, total: 19 µs
Wall time: 38.4 µs


,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity
6601,93_993911,93_0,2020-10-06,Jeff,D,Hurne,59,05/16/1961,3200,delmar blvd,NaN,chambersburg,PA,15241,Carceral,Institutionalized group quarters population,Male,White
7175,93_995927,93_0,2020-10-06,Jeff,E,Fitzgerald,49,12/23/1970,3200,delmar blvd,NaN,chambersburg,PA,15241,Carceral,Institutionalized group quarters population,Male,White
1831,99_740422,99_297529,2020-12-01,Jeff,NaN,Smith,56,01/02/1964,NaN,hubbard ln,NaN,pillow,PA,19021,Household,Opposite-sex spouse,Male,White
518,103_125897,103_50621,2020-04-21,Jeff,D,Farmer,58,05/02/1961,1121,w huntingdon st,NaN,fort myers,FL,32333,NaN,Sibling,Male,White
2038,103_847522,103_340490,2020-12-29,Jeff,R,Harding,55,03/16/1965,13043,bruce circle,NaN,brandon,MS,38632,Household,Reference person,Male,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,9740_407666,9740_163996,2020-06-16,Jeff,L,Adkins,43,10/22/1976,4116,macintyre la,NaN,marshalltown,IA,50125,Household,Opposite-sex unmarried partner,Male,White
1862,9740_854454,9740_343489,2020-12-01,Jeff,L,Osorio,48,05/17/1972,245,thorncliff road,NaN,fremont,CA,92457,Household,Opposite-sex spouse,Male,Latino
146,9871_36245,9871_14507,2020-02-25,Jeff,R,Knox,40,12/29/1979,1,n burrage ave,NaN,ardmore,OK,74055,Household,Reference person,Male,Black
871,9971_579995,9971_233044,2020-06-16,Jeff,NaN,Angeles Perezsanchez,58,11/30/1961,NaN,deerland,NaN,phoenix,AZ,85028,Household,Sibling,Male,Latino


In [10]:
people_named_jeff_pandas.memory_usage(deep=True)

Index                               1120
simulant_id                         9494
household_id                        9421
survey_date                         1120
first_name                          8540
middle_initial                      8016
last_name                           8905
age                                 8232
date_of_birth                       9346
street_number                       8196
street_name                         9672
unit_number                         4792
city                                9144
state                               3149
zipcode                             8620
housing_type                        9161
relationship_to_reference_person    1568
sex                                  264
race_ethnicity                       589
dtype: int64

In [13]:
people_named_jeff_pandas.memory_usage(deep=True).sum()

119349

In [14]:
%%time
usa_acs_pandas = usa_acs.compute()

2024-05-21 16:31:05.607 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function leave_blank on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 16:31:05.722 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function leave_blank on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 16:31:05.861 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function write_wrong_digits on column unit_number. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-21 16:31:06.035 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function write_wrong_digits on column unit_number. This is likely due to a combination of the 

CPU times: user 12.6 s, sys: 4.84 s, total: 17.5 s
Wall time: 3min 32s


In [15]:
%%time
usa_acs_pandas

CPU times: user 8 µs, sys: 4 µs, total: 12 µs
Wall time: 24.3 µs


,simulant_id,household_id,survey_date,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity
1,28_31572,28_12763,2020-01-28,Nia,B,Gomez,12,03/26/2007,51,stancliff rd,NaN,farmington,MN,55104,Household,Biological child,Female,Latino
3,28_66965,28_27014,2020-01-28,Kathleen,E,Nies,62,03/31/1957,2513,broadway street,NaN,florence,MS,38732,Household,Reference person,Female,White
4,28_103770,28_41938,2020-01-28,David,A,Wilson,67,05/07/1952,810,braxton pl,NaN,fruita,CO,80016,Household,Reference person,Male,Black
7,28_107747,28_43578,2020-01-28,Jessica,J,Palmer,35,08/15/1984,1000,aintree ct,NaN,ardmore,OK,73439,Household,Opposite-sex spouse,Female,White
10,28_138628,28_55890,2020-01-28,Rose,L,Mckenney,83,07/19/1936,405,ne tillamook st,NaN,jacksonville,FL,NaN,Household,Reference person,Female,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,9971_948430,9971_381034,2020-12-29,Jade,A,Cupp,13,10/04/2007,22,sw 147th st,NaN,aitkin,MN,55712,Household,Biological child,Female,White
1992,9971_949911,9971_381620,2020-12-29,James,C,Gaffney,50,10/23/1970,8027,deumant terrace,NaN,upper uwchlan,PA,15202,Household,Reference person,Male,White
1998,9971_1004954,9971_298253,2020-12-29,Kaliyah,I,Stofa,1,04/16/2019,505,e 7th st,NaN,seattle,WA,98125,Household,Biological child,Female,White
1999,9971_1019196,9971_123975,2020-12-29,Amaya,A,Lam,0,02/19/2020,541,coyote creek way,NaN,savannah,GA,30113,Household,Other nonrelative,Female,Asian


In [16]:
%%time
usa_acs_pandas.memory_usage(deep=True)

CPU times: user 653 ms, sys: 30.1 ms, total: 683 ms
Wall time: 680 ms


Index                                2809592
simulant_id                         23811469
household_id                        23664510
survey_date                          2809592
first_name                          21988152
middle_initial                      20276368
last_name                           22215595
age                                 20586525
date_of_birth                       23404741
street_number                       20692096
street_name                         24132304
unit_number                         11887967
city                                23032951
state                                 354208
zipcode                             21670084
housing_type                        23054874
relationship_to_reference_person      352627
sex                                   351323
race_ethnicity                        351648
dtype: int64

In [17]:
usa_acs_pandas.memory_usage(deep=True).sum() / 1e6

287.446626

In [20]:
%%time
usa_ssa = psp.generate_social_security(usa_data_dir, engine='dask')
usa_ssa

CPU times: user 1min 14s, sys: 18 s, total: 1min 32s
Wall time: 35min 3s


,simulant_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
npartitions=334,,,,,,,,,
,object,object,object,object,object,object,category[unknown],category[unknown],object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [22]:
%%time
usa_ssa.head()

2024-05-23 12:26:59.558 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function leave_blank on column ssn. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-23 12:27:01.329 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function copy_from_household_member on column ssn. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-23 12:27:02.525 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function write_wrong_digits on column ssn. This is likely due to a combination of the configuration noise levels and the simulated population data.
2024-05-23 12:27:06.114 | DEBUG    | pseudopeople.entity_types:__call__:115 - No cells chosen to noise for noise function make_ocr_errors on column ssn. This is likely due to a combination of the configuration noise 

CPU times: user 1.01 s, sys: 0 ns, total: 1.01 s
Wall time: 20.2 s


,simulant_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
0,28_475430,457-11-9330,Emily,Isabel,Cotton,19191204,Female,creation,99191204
1,28_992096,681-89-0695,Chester,Edmond,Maccracken,NaN,Male,creation,19200412
2,28_998472,237-02-4860,Caroline,Dorothy,Abney,19200501,Female,creation,19200501
3,28_985500,436-83-5692,NaN,Mary,Ross,19200709,Female,creation,19200709
4,28_238797,828-49-7950,Margaret,Lorene,Suleman,19200717,Female,creation,19202727
